### Configuração do ambiente

Para esta atividade de introdução, iremos utilizar um **modelo R-CNN pré-treinado**.

Iremos clonar o repositório do Mask_RCNN para TensorFlow 2 do Github.

[Github](https://github.com/alsombra/Mask_RCNN-TF2)

Descomentar essa sessão na primeira vez que rodar o código para configurar o ambiente.

In [1]:
#!git clone https://github.com/alsombra/Mask_RCNN-TF2

Cloning into 'Mask_RCNN-TF2'...


Precisaremos de executar o arquivo **setup.py** dentro do repositório Mask_RCNN-TF2 para realizar a instalação da Mask R-CNN.

Precisaremos chegar até o diretório via código e executar este arquivo.

O caracter % (percentagem) indica que o comando a seguir é em linux. Necessário para não dar warnings (avisos) no código.

In [2]:
# Comando linux que indica o diretório atual que estamos
#%pwd

'c:\\Users\\DIOGO-PC\\Desktop\\Estudos\\Computer_Vision_Learn\\Segmentação_de_Imagens\\Segmentação_de_Instâncias'

In [3]:
#%cd Mask_RCNN-TF2/

c:\Users\DIOGO-PC\Desktop\Estudos\Computer_Vision_Learn\Segmentação_de_Imagens\Segmentação_de_Instâncias\Mask_RCNN-TF2


In [4]:
# Agora estamos no diretório Mask_RCNN_TF2
#%pwd

'c:\\Users\\DIOGO-PC\\Desktop\\Estudos\\Computer_Vision_Learn\\Segmentação_de_Imagens\\Segmentação_de_Instâncias\\Mask_RCNN-TF2'

In [5]:
#!pip install -r requirements.txt

In [6]:
#!python setup.py install

c:\Users\DIOGO-PC\Desktop\Estudos\Computer_Vision_Learn\Segmentação_de_Imagens\Segmentação_de_Instâncias\Mask_RCNN-TF2\setup.py:9: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
c:\Users\DIOGO-PC\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\dist.py:488: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separated 'description-file' will not be supported in future
        versions. Please use the underscore name 'description_file' instead.

        By 2025-Mar-03, you need to update your project and remove deprecated calls
        or your builds will no longer be supported.

        See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
        ***********************************************************************

Retornando para o diretório raiz do projeto

In [7]:
#%cd ..

c:\Users\DIOGO-PC\Desktop\Estudos\Computer_Vision_Learn\Segmentação_de_Imagens\Segmentação_de_Instâncias


### Importação das bibliotecas

In [8]:
import os
import sys
import random
import math
import cv2
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import keras

In [9]:
# Armazenar o diretório raiz da onde se encontra o repositório
ROOT_DIR = os.path.abspath('./Mask_RCNN-TF2') # Concatenando o diretório atual com o Mask RCNN
ROOT_DIR

'c:\\Users\\DIOGO-PC\\Desktop\\Estudos\\Computer_Vision_Learn\\Segmentação_de_Imagens\\Segmentação_de_Instâncias\\Mask_RCNN-TF2'

In [10]:
path_exists = False

for i in sys.path:
    if i == ROOT_DIR:
        path_exists = True
        break

# Adicionando o caminho para o repositório do Mask R-CNN caso o path não exista
if not path_exists:
    sys.path.append(ROOT_DIR)

sys.path

['c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\DLLs',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\lib',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310',
 '',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\mask_rcnn-2.1-py3.10.egg',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\Pythonwin',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\setuptools\\_vendor',
 'c:\\Users\\DIOGO-PC\\Desktop\\Estudos\\Computer_Vision_Learn\\Segmentação_de_Ima

Como adicionamos o caminho do repositório no path, conseguimos realizar a importação de alguns arquivos que estão dentro do repositório e que iremos utilizar.

In [11]:
from mrcnn import utils
from mrcnn import visualize
import mrcnn.model as modellib

Iremos adicionar outro path, no caso, dos exemplos (samples)

In [12]:
ROOT_DIR_COCO_SAMPLES = os.path.abspath('./Mask_RCNN-TF2/samples/coco/')

path_coco_samples_exists = False

for i in sys.path:
    if i == ROOT_DIR_COCO_SAMPLES:
        path_coco_samples_exists = True
        break


# Adicionando o caminho para os exemplos caso não exista
if not path_coco_samples_exists:
    sys.path.append(ROOT_DIR_COCO_SAMPLES)

sys.path

['c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\DLLs',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\lib',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310',
 '',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\mask_rcnn-2.1-py3.10.egg',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\Pythonwin',
 'c:\\Users\\DIOGO-PC\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\setuptools\\_vendor',
 'c:\\Users\\DIOGO-PC\\Desktop\\Estudos\\Computer_Vision_Learn\\Segmentação_de_Ima

Instalação do pycocotools para importação do coco.

In [13]:
#!pip install pycocotools

In [14]:
# Caso o path for adicionado corretamente, não dará erros na importação
import coco

In [15]:
MODEL_DIR = os.path.join(ROOT_DIR, 'logs') # Este diretório só existirá após a rede neural estiver em funcionamento
IMAGE_DIR = os.path.join(ROOT_DIR, 'images') # Contém várias imagens de teste para utilizarmos

print(MODEL_DIR)
print(IMAGE_DIR)

c:\Users\DIOGO-PC\Desktop\Estudos\Computer_Vision_Learn\Segmentação_de_Imagens\Segmentação_de_Instâncias\Mask_RCNN-TF2\logs
c:\Users\DIOGO-PC\Desktop\Estudos\Computer_Vision_Learn\Segmentação_de_Imagens\Segmentação_de_Instâncias\Mask_RCNN-TF2\images
